# Import Necessary Libraries

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, InputLayer, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split

Creating a training list with a sample data of 10,000 frames, limiting the number of frames as computing power in local system is limited and prone to crashing. 

In [2]:
train = pd.read_csv('train_new.csv')
train = train.sample(n=10000).reset_index(drop=True)
train.head()

,image,class
0,v_HandstandWalking_g11_c03.avi_frame5.jpg,HandstandWalking
1,v_SoccerJuggling_g18_c03.avi_frame4.jpg,SoccerJuggling
2,v_Drumming_g16_c07.avi_frame7.jpg,Drumming
3,v_PlayingTabla_g08_c01.avi_frame2.jpg,PlayingTabla
4,v_TaiChi_g23_c02.avi_frame7.jpg,TaiChi


Creating an array of images using keras.Image, frames are loaded and converted into a numpy array for training purposes

In [3]:
# creating an empty list
train_image = []

# for loop to read and store frames
for i in tqdm(range(train.shape[0])):
    # loading the image and keeping the target size as (224,224,3)
    img = image.load_img('train_1/'+train['image'][i], target_size=(224,224,3))
    # converting it to array
    img = image.img_to_array(img)
    # normalizing the pixel value
    img = img/255
    # appending the image to the train_image list
    train_image.append(img)
    
# converting the list to numpy array
X = np.array(train_image)

# shape of the array
X.shape

100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:46<00:00, 217.35it/s]


(10000, 224, 224, 3)

Creating Class labels and spliting data into training & testing datasets.
Base model VGG16 is loaded and weights of 'imagenet' is used for training

In [4]:
# separating the target
y = train['class']

# creating the training and validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify = y)

In [5]:
# creating dummies of target variable for train and validation set
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [6]:
# creating the base model of pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False)

In [7]:
# extracting features for training frames
X_train = base_model.predict(X_train)
X_train.shape

(8000, 7, 7, 512)

In [8]:
# extracting features for validation frames
X_test = base_model.predict(X_test)
X_test.shape

(2000, 7, 7, 512)

In [9]:
# reshaping the training as well as validation frames in single dimension
X_train = X_train.reshape(8000, 7*7*512)
X_test = X_test.reshape(2000, 7*7*512)

In [10]:
# normalizing the pixel values
max = X_train.max()
X_train = X_train/max
X_test = X_test/max

In [11]:
# shape of images
X_train.shape

(8000, 25088)

# Define Model Architecture & Model Training
Create model architecture & begin to train the model. Best model weights are saved under 'weight.hdf5', this can be used to model evaluation. 

In [12]:
#defining the model architecture
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(25088,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(101, activation='softmax'))

In [14]:
# defining a function to save the weights of best model
from tensorflow.keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [15]:
# compiling the model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [16]:
# training the model
model.fit(X_train, y_train, epochs=200, validation_data=(X_test, y_test), callbacks=[mcp_save], batch_size=128)

Epoch 1/200
63/63 [==============================] - 3s 37ms/step - loss: 4.6272 - accuracy: 0.0161 - val_loss: 4.5791 - val_accuracy: 0.0260
Epoch 2/200
63/63 [==============================] - 1s 18ms/step - loss: 4.5339 - accuracy: 0.0246 - val_loss: 4.4100 - val_accuracy: 0.0380
Epoch 3/200
63/63 [==============================] - 1s 18ms/step - loss: 4.3420 - accuracy: 0.0419 - val_loss: 4.1762 - val_accuracy: 0.0605
Epoch 4/200
63/63 [==============================] - 1s 17ms/step - loss: 4.1596 - accuracy: 0.0648 - val_loss: 3.9516 - val_accuracy: 0.1170
Epoch 5/200
63/63 [==============================] - 1s 17ms/step - loss: 3.9064 - accuracy: 0.1041 - val_loss: 3.5602 - val_accuracy: 0.1710
Epoch 6/200
63/63 [==============================] - 1s 18ms/step - loss: 3.6652 - accuracy: 0.1349 - val_loss: 3.3053 - val_accuracy: 0.2140
Epoch 7/200
63/63 [==============================] - 1s 18ms/step - loss: 3.4445 - accuracy: 0.1667 - val_loss: 3.0885 - val_accuracy: 0.2505
Epoch 

63/63 [==============================] - 1s 18ms/step - loss: 1.0255 - accuracy: 0.6996 - val_loss: 1.3613 - val_accuracy: 0.6475
Epoch 116/200
63/63 [==============================] - 1s 18ms/step - loss: 1.0379 - accuracy: 0.6880 - val_loss: 1.3695 - val_accuracy: 0.6445
Epoch 117/200
63/63 [==============================] - 1s 18ms/step - loss: 1.0485 - accuracy: 0.6950 - val_loss: 1.3385 - val_accuracy: 0.6545
Epoch 118/200
63/63 [==============================] - 1s 18ms/step - loss: 1.0132 - accuracy: 0.7041 - val_loss: 1.3608 - val_accuracy: 0.6540
Epoch 119/200
63/63 [==============================] - 1s 18ms/step - loss: 1.0359 - accuracy: 0.7005 - val_loss: 1.3299 - val_accuracy: 0.6475
Epoch 120/200
63/63 [==============================] - 1s 18ms/step - loss: 1.0354 - accuracy: 0.7007 - val_loss: 1.3400 - val_accuracy: 0.6490
Epoch 121/200
63/63 [==============================] - 1s 18ms/step - loss: 1.0267 - accuracy: 0.6989 - val_loss: 1.3447 - val_accuracy: 0.6680
Epoch 

Epoch 172/200
63/63 [==============================] - 1s 18ms/step - loss: 0.8959 - accuracy: 0.7358 - val_loss: 1.4488 - val_accuracy: 0.6530
Epoch 173/200
63/63 [==============================] - 1s 18ms/step - loss: 0.9200 - accuracy: 0.7375 - val_loss: 1.4618 - val_accuracy: 0.6485
Epoch 174/200
63/63 [==============================] - 1s 18ms/step - loss: 0.8758 - accuracy: 0.7533 - val_loss: 1.4779 - val_accuracy: 0.6610
Epoch 175/200
63/63 [==============================] - 1s 18ms/step - loss: 0.9461 - accuracy: 0.7314 - val_loss: 1.5702 - val_accuracy: 0.6470
Epoch 176/200
63/63 [==============================] - 1s 18ms/step - loss: 0.9123 - accuracy: 0.7339 - val_loss: 1.4427 - val_accuracy: 0.6495
Epoch 177/200
63/63 [==============================] - 1s 18ms/step - loss: 0.9115 - accuracy: 0.7427 - val_loss: 1.4601 - val_accuracy: 0.6405
Epoch 178/200
63/63 [==============================] - 1s 18ms/step - loss: 0.8946 - accuracy: 0.7433 - val_loss: 1.4413 - val_accuracy: